<a href="https://colab.research.google.com/github/lovegaoshi/ipynb/blob/main/inaseg_google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install inaSpeechSegmenter ShazamAPI youtube-dl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 12.3 MB/s 
     |████████████████████████████████| 89 kB 9.5 MB/s 
     |████████████████████████████████| 78 kB 8.4 MB/s 
     |████████████████████████████████| 65 kB 4.1 MB/s 
     |████████████████████████████████| 130 kB 73.3 MB/s 


In [ ]:
!apt-get install aria2 ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  aria2 libc-ares2
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,274 kB of archives.
After this operation, 4,912 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libc-ares2 amd64 1.14.0-1ubuntu0.1 [37.5 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 aria2 amd64 1.33.1-1 [1,236 kB]
Fetched 1,274 kB in 1s (2,135 kB/s)
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 155639 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.14.0-1ubuntu0.1_amd64.deb ...
Unpacking libc-ares2:amd64 (1.14.0-1ubu

In [ ]:
!youtube-dl -f bestaudio  --external-downloader aria2c --external-downloader-args "-x 16 -s 16 -k 1M" https://www.youtube.com/watch?v=jCZD_sUuDWE&list=PLHBp-xPk1iMqUl-kfXFDBe1NVQLyipQAW

[youtube] jCZD_sUuDWE: Downloading webpage
[download] Destination: 「雨」しばり歌枠  _ Rain SONG only SINGING STREAM 【BOOGEY VOXX _ Ci】-jCZD_sUuDWE.webm

06/25 04:55:14 [NOTICE] Downloading 1 item(s)
 *** Download Progress Summary as of Sat Jun 25 04:56:14 2022 *** 
=
[#f684d2 48MiB/61MiB(78%) CN:16 DL:832KiB ETA:16s]
FILE: /content/「雨」しばり歌枠  _ Rain SONG only SINGING STREAM 【BOOGEY VOXX _ Ci】-jCZD_sUuDWE.webm.part
-


06/25 04:56:32 [NOTICE] Download complete: /content/「雨」しばり歌枠  _ Rain SONG only SINGING STREAM 【BOOGEY VOXX _ Ci】-jCZD_sUuDWE.webm.part

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
f684d2|OK  |   810KiB/s|/content/「雨」しばり歌枠  _ Rain SONG only SINGING STREAM 【BOOGEY VOXX _ Ci】-jCZD_sUuDWE.webm.part

Status Legend:
(OK):download completed.
[aria2c] Downloaded 64361095 bytes
[download] 100% of 61.38MiB in 01:17


In [ ]:
from inaSpeechSegmenter import Segmenter
import subprocess, os, glob
from threading import Thread
class TimestampMismatch(Exception):
    pass

# select a media to analyse
# any media supported by ffmpeg may be used (video, audio, urls)
import json, shutil
def segment(media, batch_size = 32, energy_ratio = 0.02):
    print('segmenting', media)
    seg = Segmenter(vad_engine='sm', detect_gender=False, energy_ratio = energy_ratio, batch_size = batch_size)
    segmentation = seg(media)
    return segmentation
def extract_music(segmentation, segment_thres = 60, segment_thres_final = 90, 
                  segment_connect = 5, start_padding = 1, end_padding = 2):
    r = []
    #bridges noEnergy segments that are likely fragmented
    for i in range(len(segmentation)-2, 0, -1):
        if segmentation[i][0] == 'noEnergy' and segmentation[i][2] - segmentation[i][1] < 2 and \
        segmentation[i-1][0] == segmentation[i+1][0]:
            segmentation[i-1] = (segmentation[i-1][0], segmentation[i-1][1], segmentation[i+1][2])
    for i in segmentation: 
        if i[0] == 'music' and i[2]-i[1] > segment_thres: r.append(['',i[1] - start_padding, i[2] + end_padding])    
    for i in range(len(r)-1, 0, -1):
        if r[i][1] - r[i-1][2] < segment_connect:
            r[i-1][2] = r[i][2]
            r[i][1] = r[i][2] + 1    
    rf = []
    for i in r:
        if i[1] < 5: continue
        if i[2]-i[1] > segment_thres_final: rf.append(i)
    return [['{}:{}:{}'.format(str(int(x[1]//3600)), 
                               str(int(x[1] % 3600 //60)), 
                               str(int(x[1] % 60)).zfill(2)), 
             '{}:{}:{}'.format(str(int(x[2]//3600)), 
                               str(int(x[2]  % 3600  //60)), 
                               str(int(x[2] % 60)).zfill(2))] for x in rf]
def extract_mah_stuff(media, segmented_stamps, outdir = None, rev = False, delimited = '/', timestamps = [], soundonly = True):
    nameswitch = False
    timestamps_ext = segmented_stamps#extract_music(segmentation)#[['{}:{}:{}'.format(str(int(x[1]//3600)), str(int(x[1] % 3600 //60)), str(int(x[1] % 60)).zfill(2)),  '{}:{}:{}'.format(str(int(x[2]//3600)), str(int(x[2]  % 3600  //60)), str(int(x[2] % 60)).zfill(2))] for x in r]
    try:
        if len(timestamps) > 0: raise FileNotFoundError()
        for i in open(r"D:\tmp\ytd\timstamp.ini", 'r', encoding='UTF-8'):
            i = i.replace('\n','').replace('」', '').replace('~「',' ').replace('「', ' ').replace(
                '『', ' ').replace('』', ' ').replace('　', ' ').replace('	', ' ').replace('\'','')
            if ' （' in i: i = i[:i.find(' （')]
            if ':' in i:
                timestamps.append([i[:i.find(' ')], i[i.find(' ')+1:]])
                #timestamps[-1][1] = timestamps[-1][1][1:]
                if delimited in timestamps[-1][1]:
                    timestamps[-1][1] = [timestamps[-1][1][:timestamps[-1][1].find('/')],
                                         timestamps[-1][1][timestamps[-1][1].find('/')+1:], ]
                    if rev: timestamps[-1][1].reverse()
                    timestamps[-1][1] = ' by '.join(timestamps[-1][1])
                #timestamps[-1][1].replace('/', ' by ')
                #timestamps[-1][1] = ' by '.join(timestamps[-1][1].split('/'))
                #timestamps[-1][1] = timestamps[-1][1].replace('-', 'by')
                while timestamps[-1][1][0] == ' ': timestamps[-1][1] = timestamps[-1][1][1:]
                while timestamps[-1][1][-1] == ' ': timestamps[-1][1] = timestamps[-1][1][:-1]
        #        nameswitch = True
            elif nameswitch:
                nameswitch = False
                timestamps[-1].append(i)
        if len(timestamps) > 0 and len(timestamps) != len(timestamps_ext): 
            #raise TimestampMismatch('check timestamp assist', timestamps, timestamps_ext)
            print('timestamp mismatch, removing missing ones...(come on, are you really gonna do this manually)')
            timestamps = fix_missing_stamps(timestamps, timestamps_ext)
            timestamps_ext = fix_missing_stamps(timestamps_ext, timestamps)
            if len(timestamps) != len(timestamps_ext): 
                raise TimestampMismatch('check timestamp assist', timestamps, timestamps_ext)
        with open(r"D:\tmp\ytd\timstamp.ini", 'w') as f:
            pass
    except FileNotFoundError:
        pass
    if len(timestamps) > 0: print('timestamp assist', [ [timestamps[x][0], timestamps_ext[x][1], timestamps[x][1], ] for x in range(len(timestamps))])
    else: print('extracted timestamps', ['{} - {}'.format(x[0], x[1]) for x in timestamps_ext])
    nameswitch = False
    file = media
    filename = file[:file.rfind('.')]
    fileext = file[len(filename):]
    filename = os.path.basename(filename)
    cmds = []
    for i in range(len(timestamps_ext)):
        oud = outdir if outdir else os.path.dirname(file)
        encoding = '-c:v copy -c:a copy'
        if soundonly:
            encoding = '-vn -ab 320k'
            fileext = '.mp3'
        try:
            prefix = timestamps[i][1].zfill(2)
            #subprocess.call('ffmpeg -i "{}" {} -c:v copy -c:a copy "{}"'.format(
            cmds.append('ffmpeg -i "{}" {} {} "{}"'.format(
            file, 
            encoding,
            '-ss {} -to {}'.format(timestamps[i][0], timestamps_ext[i][1],),
            os.path.join(oud, filename + '_{}_{}'.format(str(i), prefix) + fileext)))
        except:
            prefix = str(i).zfill(2)
            cmds.append('ffmpeg -i "{}" {} {} "{}"'.format(
                file, 
                encoding,
                '-ss {} -to {}'.format(timestamps_ext[i][0], timestamps_ext[i][1],),
                os.path.join(oud, filename + '_' + prefix + fileext)
        ))
    k = [Thread(target=ffmpeg, args=(x,)) for x in cmds]
    for i in k: i.start()
    k[-1].join()

import os, subprocess, time, multiprocessing

def concurrent_ffmpeg(target = 'ffmpeg'):
    return 0

def ffmpeg(cmd, concurrent_limit = multiprocessing.cpu_count(), wait = True):
    while concurrent_ffmpeg() > concurrent_limit:
        time.sleep(5)
    process = subprocess.Popen(cmd, shell=True)
    if wait: process.wait()
    return 1
from difflib import SequenceMatcher as SM
def fuzzy_match_my_file(fname):
    ext = fname[fname.rfind('.'):]
    r = []
    for i in glob.glob(os.path.join(
        os.path.dirname(fname),
        '*' + ext
    )):
        r.append([
            i,
            SM(isjunk=None, a = os.path.basename(fname), b = os.path.basename(i)).ratio()
        ])
    return r
    pass

saved_timestamp = None

    
def timestamp2sec(timestamp):
    # hurr durr...
    timestamp = timestamp.split(':')
    timestamp.reverse()
    seconds = 0
    for i in range(len(timestamp)): seconds += int(float(timestamp[i])) * pow(60, i)
    return seconds

def is_stamp_missing(stamp, stamps, secrange = 40):
    stamp_sec = timestamp2sec(stamp[0])
    for i in stamps:
        i_sec = timestamp2sec(i[0])
        if i_sec > stamp_sec + secrange: return True
        if abs(i_sec-stamp_sec) < secrange: return False
    return True

def fix_missing_stamps(stamps, stamps2):
    r = []
    for i in stamps:
        if not is_stamp_missing(i, stamps2): r.append(i)
        else: print(i, 'is missing and gone(puff)')
    return r

import re

In [ ]:
media = '/content/「雨」しばり歌枠  _ Rain SONG only SINGING STREAM 【BOOGEY VOXX _ Ci】-jCZD_sUuDWE.webm'
extract_mah_stuff(media, extract_music(segment(media)))

segmenting /content/「雨」しばり歌枠  _ Rain SONG only SINGING STREAM 【BOOGEY VOXX _ Ci】-jCZD_sUuDWE.webm


/usr/local/lib/python3.7/dist-packages/pyannote/algorithms/utils/viterbi.py:88: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for e, c in six.moves.zip(emission.T, consecutive)
/usr/local/lib/python3.7/dist-packages/pyannote/algorithms/utils/viterbi.py:97: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for e, c in six.moves.zip(constraint.T, consecutive)
/usr/local/lib/python3.7/dist-packages/inaSpeechSegmenter/segmenter.py:61: RuntimeWarning: invalid value encountered in subtract
  data = (data - np.mean(data, axis=1).reshape((len(data), 1))) / np.std(data, axis=1).reshape((len(data), 1))
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:23

extracted timestamps ['0:11:15 - 0:16:10', '0:19:55 - 0:23:17', '0:27:39 - 0:31:29', '0:37:05 - 0:41:12', '0:46:32 - 0:49:45', '0:55:38 - 0:58:54']
